<a href="https://colab.research.google.com/github/denisecammarota/icgpu/blob/main/Clase2_Tabulacion_MatMul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%html
<marquee style='width: 100%; color: blue;'><b>ICNPG2021 en Google Colaboratory-Instituto Balseiro-Clase 1 </b></marquee>


# Preparativos para programar CUDA C/C++ en google colabs

Hola!, Bueno, aqui va un ejemplito de como correr codigo CUDA C/C++ en colabs
[[1]](https://https://www.wikihow.com/Run-CUDA-C-or-C%2B%2B-on-Jupyter-(Google-Colab).

No se olviden de Runtime-> Change Runtime Type -> GPU. Para que funque a cada linea la tienen que ejecutar con un Shift-Enter o Ctrl-Enter o el botoncito de play.

miremos que version de nvcc tenemos

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


A ver que placa nos toco...

In [ ]:
!nvidia-smi

Thu Apr 15 12:52:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

lindas GPUs!!. Ahora, para poder correr Cuda C/C++ instalamos un plugin

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-si2ow4d2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-si2ow4d2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=505e0acb0646a16a474a281f3e63cb87d4a1ccc80a4894f1cc5174c218716f60
  Stored in directory: /tmp/pip-ephem-wheel-cache-ynit1yxe/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


y luego:

In [ ]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


Listo!, con eso ya podemos correr codigo CUDA C/C++ en el notebook Jupyter.

Para terminar, es conveniente montar nuestro google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Tabulación de un array en CPU y en GPU


Completar y corregir el código siguiente para tabular un array en host y en device de cualquier tamaño. chequer la correctitud.

In [ ]:
%%cu
// solucion en la cpu

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <ctime>
#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/Codigos_de_las_Clases/common/gpu_timer.h"
#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/Codigos_de_las_Clases/common/cpu_timer.h"

#define SIZE	1024
#define NVECES	1

__device__ __host__ 
float MiFuncion(int i){
    return sin(2*M_PI*i/10.0);    
}

__global__ void Tabular(float *d_c, int n)
{
        // indice de thread mapeado a indice de array 
        int i = blockIdx.x * blockDim.x + threadIdx.x;

        //COMPLETAR PARA QUE c[i]=MiFuncion(i)
        //ASEGURARSE DE QUE TODO EL ARRAY ESTE TABULADO CON LA GRILLA LANZADA
        //Y DE QUE NO SE ACCEDAN POSICIONES ILEGALES
}

int main(int argc, char **argv)
{
        int N;

        if(argc==2) N=atoi(argv[1]);
        else N=SIZE;

        // punteros a memoria de host
        float *c;

        // alocacion memoria de host
        c = (float *)malloc(N*sizeof(float));

        /////////////////////// TABULACION EN CPU 
        // timer para gpu...
        cpu_timer RelojCPU;
        RelojCPU.tic();

        // suma paralela en el device
        for(int i=0;i<NVECES;i++){
            for(int n=0;n<N;n++){
              c[n]=MiFuncion(n);                
            }
        }

        // verificacion del resultado
        for( int i = 0; i < N; ++i){
                if(i<10) printf("c[%d] = %f\n", i, c[i]);
        }

        // milisegundos transcurridos
        printf("Tabular en CPU, N= %d t= %lf ms\n", N, RelojCPU.tac());    

        /////////////////////// TABULACION EN GPU 

        // punteros a memoria de device
        float *d_c;

        // alocacion memoria de device
        // COMPLETAR

        // grilla de threads suficientemente grande...
        // COMPLETAR LA CONFIGURACION DE LA GRILLA USANDO dim3 nThreads, nBlocks
        // PARA QUE SE PUEDA TABULAR UN ARRAY DE CUALQUIER TAMANIO
        dim3 nThreads(256); // CORREGIR
        dim3 nBlocks(1); // CORREGIR

        // suma paralela en el device: WARMING UP
        Tabular<<< nBlocks, nThreads >>>(d_c, N);

        // timer para gpu...
        gpu_timer RelojGPU;
        RelojGPU.tic();

        // suma paralela en el device
        for(int i=0;i<NVECES;i++)
        Tabular<<< nBlocks, nThreads >>>(d_c, N);

        // milisegundos transcurridos
        printf("GPU: Tabular<<< %d, %d >>>, N= %d, t= %lf ms\n", nBlocks.x, nThreads.x, N, RelojGPU.tac());    

        // copia (solo del resultado) del device a host
        // COMPLETAR PARA TRAER LOS DATOS DE LA GPU, PONERLOS EN c[]
        // .....

        // verificacion del resultado
        for( int i = 0; i < N; ++i){
                if(i<10) printf("c[%d] = %f\n", i, c[i]);
        }

        // liberacion memoria de host
        free(c);

        // liberacion memoria de device
        cudaFree(d_c);

        return 0;
}


# ¿Que placa soy?

In [ ]:
%%cu
#include <stdio.h>

int main(int argc, char **argv)
{
	cudaDeviceProp deviceProp;

	int deviceCount = 0;
    	cudaError_t error_id = cudaGetDeviceCount(&deviceCount);


	printf("En este nodo hay %d placas\n\n",deviceCount);
	for(int dev=0;dev<deviceCount;dev++){
	    	cudaSetDevice(dev);
    		cudaGetDeviceProperties(&deviceProp, dev);
    		printf("Hola!, yo soy [Device %d: \"%s\"], tu acelerador grafico personal\n", dev, deviceProp.name);
	}

	int dev; cudaGetDevice(&dev);
	printf("\nle asigno la device %d, que esta desocupada\n", dev);

	return 0;
}


# Multiplicacion de Matrices

Completar para comparar la multiplicacion de matrices en paralelo (simple) y serial


In [ ]:
%%cu
#include<stdio.h>
#include <cuda_runtime.h>
#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/Codigos_de_las_Clases/common/gpu_timer.h"
#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/Codigos_de_las_Clases/common/cpu_timer.h"

// Matrices are stored in row-major order:
// M(row, col) = *(M.elements + row * M.width + col)
typedef struct {
    int width;
    int height;
    float* elements;
} Matrix;

// Thread block size
#define BLOCK_SIZE 16

// Forward declaration of the matrix multiplication kernel
__global__ void MatMulKernel(const Matrix, const Matrix, Matrix);

// Matrix multiplication - Host code
// Matrix dimensions are assumed to be multiples of BLOCK_SIZE
void MatMul(const Matrix A, const Matrix B, Matrix C)
{
    // Load A and B to device memory
    Matrix d_A;
    d_A.width = A.width; d_A.height = A.height;
    size_t size = A.width * A.height * sizeof(float);
    cudaMalloc(&d_A.elements, size);
    cudaMemcpy(d_A.elements, A.elements, size,
               cudaMemcpyHostToDevice);
    Matrix d_B;
    d_B.width = B.width; d_B.height = B.height;
    size = B.width * B.height * sizeof(float);
    cudaMalloc(&d_B.elements, size);
    cudaMemcpy(d_B.elements, B.elements, size,
               cudaMemcpyHostToDevice);

    // Allocate C in device memory
    Matrix d_C;
    d_C.width = C.width; d_C.height = C.height;
    size = C.width * C.height * sizeof(float);
    cudaMalloc(&d_C.elements, size);

    // Invoke kernel
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid(B.width / dimBlock.x, A.height / dimBlock.y);
    MatMulKernel<<<dimGrid, dimBlock>>>(d_A, d_B, d_C);

    // Read C from device memory
    cudaMemcpy(C.elements, d_C.elements, size,
               cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_A.elements);
    cudaFree(d_B.elements);
    cudaFree(d_C.elements);
}

// Matrix multiplication kernel called by MatMul()
__global__ void MatMulKernel(Matrix A, Matrix B, Matrix C)
{
    // Each thread computes one element of C
    // by accumulating results into Cvalue
    float Cvalue = 0;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    for (int e = 0; e < A.width; ++e)
        Cvalue += A.elements[row * A.width + e]
                * B.elements[e * B.width + col];
    C.elements[row * C.width + col] = Cvalue;
}


// Matrix multiplication in cpu
void MatMulcpu(Matrix A, Matrix B, Matrix C)
{
//TODO: completar para que C=A*B en la cpu
}

#define DIM	1024
#define  IDX2C(i,j,ld) (((j)*(ld))+( i ))

int main(int argc, char **argv)
{
	//TODO: completar
	// (1) alocar e inicializar A y B en host
	// (2) chequear y cronometrar MatMul
	// (3) chequear y cronometrar MatMulcpu
	// (4) Comparar CPU vs GPU para distintos tamaños de matriz

        return 0;
}